<a href="https://colab.research.google.com/github/JungAh12/Everyone_TF2.0/blob/master/TF2_0_for_experts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tensorflow import

In [1]:
!pip install -q tensorflow-gpu==2.0.0-rc1
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

     |████████████████████████████████| 380.5MB 38kB/s 
     |████████████████████████████████| 4.3MB 33.8MB/s 
     |████████████████████████████████| 501kB 50.3MB/s 


#MNIST 데이터셋 준비하기

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train,x_test = x_train/255.0, x_test/255.0    # normalization

print(x_train.shape)
print(x_test.shape)

# 채널 차원 추가하기(tf.newaxis = 차원 추가)
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000, 28, 28, 1)
(10000, 28, 28, 1)


#데이터셋 섞고 배치 생성

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)    # 함수 체이닝으로 
test_dataset = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).batch(32)

#Kears 사용하여 모델 생성

In [0]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

# optimizer, loss 설정

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

#loss, performance metric 설정

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

#초보자용 튜토리얼과의 차이

초보자용 코드

```python
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
```

전문가용 코드
```python
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
```

PyTorch와 똑같아 보이는데..??

#모델 훈련

`train_step`함수는 example과 label을 데이터로 받는다.
`with tf.GradientTape() as tape`로 GradientTape를 사용한다. 

GradientTape에 대해서는 추후에..

In [0]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

#모델 테스트

In [0]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)


In [16]:
EPOCHS = 5  #학습 반복 수

for epoch in range(EPOCHS):
    for images, labels in train_dataset:    #학습 배치 꺼내서 학습함
        train_step(images, labels)
    
    for test_images, test_labels in test_dataset:   #테스트 배치 꺼내서 학습
        test_step(test_images, test_labels)

    template = 'epoch:{}, loss:{}, acc:{}, t_loss:{}, t_acc:{}'
    #각 epoch 당 학습 손실값과 정확도값, 테스트 손실값, 정확도값
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result()*100,
                          test_loss.result(),
                          test_accuracy.result()*100))

epoch:1, loss:0.13788779079914093, acc:95.79499816894531, t_loss:0.07063054293394089, t_acc:97.68000030517578
epoch:2, loss:0.0915590301156044, acc:97.18582916259766, t_loss:0.0605606772005558, t_acc:98.01499938964844
epoch:3, loss:0.06845221668481827, acc:97.89999389648438, t_loss:0.060715511441230774, t_acc:98.02333068847656
epoch:4, loss:0.0546390525996685, acc:98.31458282470703, t_loss:0.06475825607776642, t_acc:98.0199966430664
epoch:5, loss:0.045788560062646866, acc:98.58100128173828, t_loss:0.06326403468847275, t_acc:98.09600067138672
